# Energy Reconstruction Using CNN

## General Notes:
This notebook is designed for regression convolutional neural networks used to reconstruct the energy of a cosmic ray primary. It assumes all input is normalized as a best-practice.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from data_tools import dataPrep, getCut, getRecoFilter, load_preprocessed
from keras.models import load_model

## Model Selection

In [ ]:
# Edit this file path to the models folder containing .h5 and .npy files for each model.
modelPrefix = os.getcwd()+'/models'

# Edit this file path to the folder containing the simulation data.
simPrefix = os.getcwd()+'/simdata'

# Option to change font size for all labels within this notebook
labelParams = {'fontsize':16}

In [ ]:
# List of available models

modelList = sorted(glob('%s/*.h5' % modelPrefix))
modelList = [os.path.basename(m)[:-3] for m in modelList]

paramList = sorted(glob('%s/*.npy' % modelPrefix))
paramList = [os.path.basename(p)[:-4] for p in paramList]

print('Available models:', sorted(set(modelList).intersection(paramList)))
print('\nModels without parameter files:', sorted(set(modelList).difference(paramList)))

In [ ]:
# Keys you want to study
keyList = []
descriptions = []

# Automatic intake of parameters from parameter files
labels, p = {}, {}
for key, description in zip(keyList, descriptions):
    labels[key] = description
    d = np.load('%s/%s.npy' % (modelPrefix, key), allow_pickle=True)
    p[key] = d.item()
    print(key, ':', p[key])

## Assessment Data

In [ ]:
# Load data, x in eight layers (q1h, q1s, q2h, q2s, t1h, t1s, t2h, t2s) and y as a dictionary with event-level parameters
x, y = load_preprocessed(simPrefix, 'assessment')

## Notes

This notebook is designed to compare multiple models for beginners in machine learning

## Model Preperation

In [ ]:
models = {}
recoE = {}

# Calculate reconstructed energies. This can take a bit, but should print out info on each key as it works
for key in keyList:

    # Comment these two lines if you want to rerun your energy reconstructions each time
    if key in models.keys():
        continue
    
    print('Working on %s...' % key)
    print(p[key])
    # Note: very sensitive to tensorflow/keras version.
    models[key] = load_model('%s/%s.h5' % (modelPrefix, key)) # Edit file path
    
    # Configure input data
    x_i, idx = dataPrep(x.copy(), y.copy(), **p[key])

    if p[key]['reco'] != None:
        nanCut = getRecoFilter(p[key]['reco'], y)
        for i, _ in enumerate(x_i):
            x_i[i] = x_i[i][nanCut[y['passed_STA5']] if p[key]['sta5'] else nanCut]

    # Models should only output energy
    if p[key]['reco'] == None: # Charge and Time
        recoE[key] = models[key].predict([x_i[...,:idx], x_i[...,idx:]]).flatten()
    else:
        if p[key]['t'] == False: # Charge and Zenith
            recoE[key] = models[key].predict([x_i[0], x_i[1]]).flatten()
        else: # Charge and Time and Zenith
            recoE[key] = models[key].predict([x_i[0][...,:idx], x_i[0][...,idx:], x_i[1]]).flatten()
    print()

## Plotting Parameters

In [ ]:
ebins = np.linspace(5, 8, 181)
evalues = (ebins[:-1] + ebins[1:]) / 2

cutNames = ['No Cut', 'Quality Cut']
ncols, nrows = len(cutNames), len(keyList)

## Energy Reconstruction Plots

### Total Energy Resolution

In [ ]:
# Plot logged energy resolution

histArgs = {'range':(-2,2), 'bins':121, 'density':True, 'histtype':'step', 'log':True, 'linewidth':4}
fig, axs = plt.subplots(figsize=(13*ncols, 8), ncols=ncols)

for i, cutName in enumerate(cutNames):
    ax = axs[i]
    for j, key in enumerate(keyList):
        cut, energy = getCut(cutName, y, p, recoE, key)
        ax.hist((recoE[key][cut] - energy), label=labels[key], **histArgs)
    ax.set_title('Energy Resolution (%s)' % cutName, **labelParams)
    ax.set_xlabel(r'$\log_{10}(E_{\mathrm{reco}}/\mathrm{GeV}) - \log_{10}(E_{\mathrm{true}}/\mathrm{GeV})$', **labelParams)
    ax.set_ylabel('Counts', **labelParams)
    ax.legend()
    ax.axvline()

In [ ]:
# Plot zoomed and unlogged energy resolution

histArgs = {'range':(-1,1), 'bins':121, 'density':True, 'histtype':'step', 'linewidth':4}
fig, axs = plt.subplots(figsize=(13*ncols, 8), ncols=ncols)

for i, cutName in enumerate(cutNames):
    ax = axs[i]
    for j, key in enumerate(keyList):
        cut, energy = getCut(cutName, y, p, recoE, key)
        ax.hist((recoE[key][cut] - energy), label=labels[key], **histArgs)
    ax.set_title('Energy Resolution (%s)' % cutName, **labelParams)
    ax.set_xlabel(r'$\log_{10}(E_{\mathrm{reco}}/\mathrm{GeV}) - \log_{10}(E_{\mathrm{true}}/\mathrm{GeV})$', **labelParams)
    ax.set_ylabel('Counts', **labelParams)
    ax.legend()
    ax.axvline()

In [ ]:
# Summary parameters

for key in keyList:
    for i, cutName in enumerate(cutNames):
        cut, energy = getCut(cutName, y, p, recoE, key)
        median, err_min, err_max = np.percentile(recoE[key][cut] - energy, (50,16,84))
        print('Energy resolution for %s (%s): %.03f +%.03f %.03f' % (key, cutName, median, err_max, err_min))
    print()

### Two-Dimensional Visualization

In [ ]:
# Plot two-dimensional visualization

np.seterr(divide = 'ignore')

fig, axs = plt.subplots(figsize=(13*ncols, 10*nrows), ncols=ncols, nrows=nrows, 
                        sharex=True, sharey=True)

for i, key in enumerate(keyList):
    for j, cutName in enumerate(cutNames):
        
        ax = axs[i, j] if len(keyList) > 1 else axs[j]
        cut, energy = getCut(cutName, y, p, recoE, key)
        
        h, xedges, yedges = np.histogram2d(recoE[key][cut], energy, bins=(ebins, ebins), normed=False, weights=None)
        # Normalize
        ntot = np.sum(h, axis=0).astype(float)
        ntot[ntot==0] = 1.
        h /= ntot
        
        # Create contours
        contourValues = [0.025, 0.16, 0.84, 0.975]
        contourList = [[] for _ in contourValues]
        for c, col in enumerate(h.transpose()):
            ccol = col.cumsum()
            for l, val in zip(contourList, contourValues):
                try: l += [np.where(ccol > val)[0][0]]
                except IndexError:
                    l += [0]
        for l in contourList:
            l.insert(0, l[0])
            if i >= len(contourList) / 2:
                l = [j+1 for j in l]     
        ax.plot(evalues, evalues, 'k', ls=':')
        for l in contourList:
            ax.step(ebins, ebins[l], color='red', linestyle='--')
        
        # Plot on a log scale
        extent = [yedges[0], yedges[-1], xedges[0], xedges[-1]]
        im = ax.imshow(np.log10(h), extent=extent, origin='lower', interpolation='none', vmin=-3.5, vmax=-0.5)
        ax.set_title('%s (%s)' % (key, cutName), **labelParams)
        ax.set_xlabel(r'$\log_{10}(E_{\mathrm{true}}/\mathrm{GeV})$', **labelParams)
        ax.set_ylabel(r'$\log_{10}(E_{\mathrm{reco}}/\mathrm{GeV})$', **labelParams)
        fig.colorbar(im, ax=ax)

### Energy Resolution as a Function of Zenith

In [ ]:
# Plot energy resolution as a function of zenith

coszbins = np.linspace(0.4,1,20)
coszvalues = (coszbins[1:]+coszbins[:-1])/2
kwargs = {'fmt':'.',
          'markersize':16,
          'elinewidth':2,
          'capsize':10,
          'capthick':2}
fig, axs = plt.subplots(figsize=(13*ncols, 10), ncols=ncols)

for i, cutName in enumerate(cutNames):
    ax = axs[i]
    for k, key in enumerate(keyList):
        
        theta = y['laputop_dir'].transpose()[0]
        theta = theta[getRecoFilter(p[key]['reco'], y) * y['passed_STA5']] if p[key]['sta5'] else theta[getRecoFilter(p[key]['reco'], y)]
        theta = np.pi - theta.astype('float')  # Define 0 degrees as overhead

        array_info = np.zeros(shape=(len(coszvalues), 3))
        cut, energy = getCut(cutName, y, p, recoE, key)
        binned_zenith = np.digitize(np.cos(theta)[cut], coszbins) - 1
        for j in range(len(coszvalues)):
            coszcut = (binned_zenith == j)
            temp_events = recoE[key][cut][coszcut]
            if len(temp_events) != 0:
                array_info[j] = np.percentile(temp_events - energy[coszcut], (50, 16, 84))

        median, err_min, err_max = np.transpose(array_info)
        ax.errorbar(coszvalues, median, yerr=(median-err_min, err_max-median), label=key, **kwargs)
    
    ax.axhline(color='k', ls='--')
    ax.set_title('Energy Resolution v. Zenith (%s)' % cutName, **labelParams)
    ax.set_xlabel(r'$\cos(\theta)$', **labelParams)
    ax.set_ylabel(r'$\log_{10}(E_{\mathrm{reco}}/\mathrm{GeV}) - \log_{10}(E_{\mathrm{true}}/\mathrm{GeV})$', **labelParams)
    ax.set_ylim(-0.5, 0.5)
    ax.legend()
    
plt.show()